## 5.2 参数管理

将介绍以下内容：

- 访问参数，用于调试、诊断和可视化；
- 参数初始化；
- 在不同模型组件间共享参数。

下面是一个具有单隐藏层的多层感知机。

In [1]:
import torch
from torch import nn

net = nn.Sequential(
    nn.Linear(4, 8),
    nn.ReLU(),
    nn.Linear(8, 1),
)
X = torch.rand(size=(2, 4))
net(X)

tensor([[0.2951],
        [0.1330]], grad_fn=<AddmmBackward0>)

### 5.2.1 参数访问

下面给可以检查第二个全连接层的参数。

In [2]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.0696,  0.1417, -0.1710,  0.3052,  0.1280, -0.0508,  0.0744, -0.3076]])), ('bias', tensor([-0.0104]))])


需要注意的是，参数名称允许唯一标识每个参数

#### 5.2.1.1 目标参数

下面以四种形式访问了参数，由于没有进行反向传播，故没有梯度。

In [3]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.0104], requires_grad=True)
tensor([-0.0104])


In [4]:
net[2].bias.grad == None

True

#### 5.2.1.2 一次性访问所有参数

In [5]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


还有一种方式可以访问。

说明其本质也就只是一个字典。

In [6]:
net.state_dict()['2.bias'].data

tensor([-0.0104])

#### 5.2.1.3 从嵌套块中收集参数

In [10]:
def block1():
    return nn.Sequential(
        nn.Linear(4, 8),
        nn.ReLU(),
        nn.Linear(8, 4),
        nn.ReLU(),
    )

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.2748],
        [-0.2748]], grad_fn=<AddmmBackward0>)

In [11]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


可以像嵌套列表一样访问它们。

In [12]:
rgnet[0][1][0].bias.data

tensor([-0.2726, -0.4690,  0.4854, -0.2682,  0.2934,  0.4651, -0.2895,  0.0172])

### 5.2.2 参数初始化

默认情况下，PyTorch会根据一个范围均匀地初始化权重和偏置矩阵，这个范围是根据输入和输出维度计算出的。

#### 5.2.2.1 内置初始化

In [13]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0033, -0.0007,  0.0046, -0.0045]), tensor(0.))

也可以将所有参数初始化为给定常数。

In [21]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

对某些块使用不同的初始化方法。

这里的`xavier_uniform_`是一种与高斯分布相异的，但是功能相同的初始化方法。

In [23]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.4827, -0.5990, -0.2386,  0.2477])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


#### 5.2.2.2 自定义初始化

下面定义了一种稍许复杂的初始化方法

In [25]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        # 如果该参数的abs小于5，则会置0
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 6.7631,  5.4274,  9.8071,  5.9419],
        [-6.2393,  0.0000, -6.8842, -0.0000]], grad_fn=<SliceBackward0>)

注意，我们始终可以直接设置参数。

In [26]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  6.4274, 10.8071,  6.9419])

### 5.2.3 参数绑定

In [27]:
# 给予共享层一个特定的名称
shared = nn.Linear(8, 8)
net = nn.Sequential(
    nn.Linear(4, 8),
    nn.ReLU(),
    shared,
    nn.ReLU(),
    shared,
    nn.ReLU(),
    nn.Linear(8, 1)
)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


上面的例子说明两个参数是完全一致且共享的。

需要注意的是，**在反向传播过程中，共享层的梯度会进行累加！！！**

### 5.2.4 小结

- 我们有几种方法可以访问、初始化和绑定模型参数。
- 我们可以使用自定义初始化方法。